In [58]:
import pandas as pd
import torch
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import Counter
import scipy
from scipy import stats
import sklearn

In [34]:
labels = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/outputs/labels_corrected.csv')

In [4]:
# 1 - head
# 2 - abdomen
# 3 - femur

labels

,index,Class,video
0,1_1_1,1,1_1
1,1_1_2,1,1_1
2,1_1_3,1,1_1
3,1_1_4,1,1_1
4,1_1_5,1,1_1
...,...,...,...
261760,708_3_75,5,708_3
261761,708_3_76,5,708_3
261762,708_3_77,5,708_3
261763,708_3_78,5,708_3


In [35]:
femur_labels = labels[labels['video'].str.split('_').str[-1] == '3']
head_labels = labels[labels['video'].str.split('_').str[-1] == '1']
abdomen_labels = labels[labels['video'].str.split('_').str[-1] == '2']

In [36]:
"""new_labels:
0 - other
1 - head non-standard plane
2 - head standard plane
3 - abdomen non-standard plane
4 - abdomen standard plane
5 - femur non standard plane
6 - femur standard plane"""

'new_labels:\n0 - other\n1 - head non-standard plane\n2 - head standard plane\n3 - abdomen non-standard plane\n4 - abdomen standard plane\n5 - femur non standard plane\n6 - femur standard plane'

In [37]:
abdomen_labels['Class'] = abdomen_labels['Class'].replace([6,2,5,1], 0)

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
abdomen_labels['Class'].value_counts()

3    105108
0      8678
4      4917
Name: Class, dtype: int64

In [39]:
femur_labels['Class'].value_counts()

5    54750
6    39610
0    28059
3     2905
1      167
4       35
2       14
Name: Class, dtype: int64

In [40]:
femur_labels['Class'] = femur_labels['Class'].replace([1,2,3,4], 0)

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
femur_labels['Class'].value_counts()

5    54750
6    39610
0    31180
Name: Class, dtype: int64

In [42]:
head_labels['Class'].value_counts()

1    13410
2     2840
0     1272
Name: Class, dtype: int64

In [43]:
"""new_labels:
0 - other
1 - head non-standard plane
2 - head standard plane
3 - abdomen non-standard plane
4 - abdomen standard plane
5 - femur non standard plane
6 - femur standard plane"""

'new_labels:\n0 - other\n1 - head non-standard plane\n2 - head standard plane\n3 - abdomen non-standard plane\n4 - abdomen standard plane\n5 - femur non standard plane\n6 - femur standard plane'

In [44]:
head_labels['Class'].value_counts()

1    13410
2     2840
0     1272
Name: Class, dtype: int64

In [48]:
#Changing labels from 5,6 to 1,2 to avoid problems with cross entropy loss
femur_labels['Class'] = femur_labels['Class'].replace(5, 1)
femur_labels['Class'] = femur_labels['Class'].replace(6, 2)

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [49]:
femur_labels['Class'].value_counts()

1    54750
2    39610
0    31180
Name: Class, dtype: int64

In [51]:
abdomen_labels['Class'] = abdomen_labels['Class'].replace(3, 1)
abdomen_labels['Class'] = abdomen_labels['Class'].replace(4, 2)

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
abdomen_labels['Class'].value_counts()

1    105108
0      8678
2      4917
Name: Class, dtype: int64

In [53]:
## Data split
def probability_mass(data):
    
    counts = Counter(data) # counting the classes
    total = sum(counts.values()) # total number of classes
    probability_mass = {k:v/total for k,v in counts.items()} # probability mass of the classes
    probability_mass = list(probability_mass.values()) # converting the dictionary to a list
    return probability_mass
    

def train_test_split(data, train_size = 0.9, precision = 0.005):
    """splitting data into train and test sets keeping the same distribution of classes using wasertein's method
    args: data - data frame containing the data
    train_size - size of the train set default
    precision - determines how close the train set size is to the train_size default 0.005 (the smaller the better, but it may take longer to generate sets)"""
    
    print('Splitting data into train and test sets...')
    
    #data = pd.read_csv(data) # loading the data
    wass_dist = 1
    videos = list(data['video'].unique()) # list of videos
    train_size = int(train_size * len(videos)) # calculating the number of videos in the train set
    while wass_dist > precision: # while the wasserstein distance is greater than 0.005
        train = random.sample(videos, train_size) # sampling the train set
        testval = [x for x in videos if x not in train] # sampling the test set
        train_set = data.loc[data['video'].isin(train)] # creating the train set
        testval_set = data.loc[data['video'].isin(testval)] # creating the test set
        test_set = testval_set.sample(frac = 0.5) # sampling the test set
        val_set = testval_set.drop(test_set.index) # sampling the validation set
        probability_mass_train = probability_mass(train_set['Class']) # calculating the probability mass of the train set
        probability_mass_test = probability_mass(test_set['Class']) # calculating the probability mass of the test set
        wass_dist = scipy.stats.wasserstein_distance(probability_mass_train, probability_mass_test) # wasserstein distance between distributions
    print('Done!')
    return train_set, val_set, test_set
    
def histogram_class_plot(data, title):
    """plotting histogram of the classes"""
    plt.figure(figsize=(10,10))
    plt.hist(data['Class'], bins = 100)
    plt.xlabel('Class')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.savefig(title + '.png')






In [54]:
abdomen_train, abdomen_val, abdomen_test = train_test_split(abdomen_labels)
femur_train, femur_val, femur_test = train_test_split(femur_labels)
head_train, head_val, head_test = train_test_split(head_labels)

Splitting data into train and test sets...
Done!
Splitting data into train and test sets...
Done!
Splitting data into train and test sets...
Done!


In [55]:
abdomen_train.to_csv('/data/kpusteln/fetal/standard_plane/class_data/abdomen_train.csv', index = False)
abdomen_val.to_csv('/data/kpusteln/fetal/standard_plane/class_data/abdomen_val.csv', index = False)
abdomen_test.to_csv('/data/kpusteln/fetal/standard_plane/class_data/abdomen_test.csv', index = False)

femur_train.to_csv('/data/kpusteln/fetal/standard_plane/class_data/femur_train.csv', index = False)
femur_val.to_csv('/data/kpusteln/fetal/standard_plane/class_data/femur_val.csv', index = False)
femur_test.to_csv('/data/kpusteln/fetal/standard_plane/class_data/femur_test.csv', index = False)

head_train.to_csv('/data/kpusteln/fetal/standard_plane/class_data/head_train.csv', index = False)
head_val.to_csv('/data/kpusteln/fetal/standard_plane/class_data/head_val.csv', index = False)
head_test.to_csv('/data/kpusteln/fetal/standard_plane/class_data/head_test.csv', index = False)

In [56]:
abdomen_train['Class'].value_counts()

1    94652
0     7906
2     4453
Name: Class, dtype: int64

In [80]:
abd_classes = abdomen_train['Class']
hd_classes = head_train['Class']
fm_classes = femur_train['Class']

In [81]:
# Class balancing
abd_class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight= 'balanced', classes = np.unique(abd_classes), 
                                                y = abd_classes)
fm_class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight= 'balanced', classes = np.unique(fm_classes),
                                                y = fm_classes)

hd_class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight= 'balanced', classes = np.unique(hd_classes),
                                                y = hd_classes)
                                                                   

In [86]:
abd_class_weights

array([4.51180538, 0.37685768, 8.01040497])